# Generating Text A Many To Many Architecture

I will create and train a text generating model which will generate the next possible 50 words for a given sequence of 10 words.

### Steps for  text generating model. 
* Step-1:  Downloaded Gutenberg's Alice's Adventures in Wonderland dataset from https://www.gutenberg.org/files/11/11-0.txt.
* Step-2:  Preprocess the text data to convert in lower case, and remove punctuation.
* Step-3:  Assign an ID to each unique word and then convert the dataset into a sequence of word IDs.
* Step-4:  Loop through the total dataset, 10 words at a time. Consider the 10 words as input and the subsequent 11th word as output. 
* Step-5:  Build and train a model, by performing embedding on top of the input word IDs and then connecting the embeddings to an LSTM, which is connected to the output layer through a hidden layer. The value in the output layer is the one-hot-encoded version of the output.
* Step-6:  Make a prediction for the subsequent word by taking a random location of word and consider the historical words prior to the location of the random word chosen.
* Step-7:  Move the window of the input words by one from the seed word's location that we chose earlier and the tenth time step word shall be the word that we predicted in the previous step.
* Step-8  Continue this process to keep generating text.

Import the modules.

In [1]:
import re
import numpy as np
from collections import Counter
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense,Activation

Using TensorFlow backend.


In [2]:
fin=open('./data/11-0.txt',encoding='utf-8-sig')
lines=[]
for line in fin:
    line = line.strip()
    #line = line.strip().lower()
    #line = line.decode("ascii","ignore")
    if(len(line)==0):
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

A sample of the dataset looks as follows.

In [3]:
text[0:200]

'Project Gutenberg’s Alice’s Adventures in Wonderland, by Lewis Carroll This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  You may copy it, give it awa'

Normalize the text to remove punctuations and convert it to lowercase.

In [4]:
text = text.lower()
text = re.sub('[^0-9a-zA-Z]+',' ',text)

A sample of the dataset looks as follows after Normalize the text.

In [5]:
text[0:200]

'project gutenberg s alice s adventures in wonderland by lewis carroll this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever you may copy it give it away or'

Assign the unique words to an index so that they can be referenced when constructing the training and test datasets

In [6]:
counts = Counter()
counts.update(text.split())
words = sorted(counts, key=counts.get, reverse=True)
nb_words = len(text.split())
word2index = {word: i for i, word in enumerate(words)}
index2word = {i: word for i, word in enumerate(words)}

Construct the input set of words that leads to an output word. 
* Note that I am considering a sequence of 10 words and trying to predict the 11th word.

In [7]:
SEQLEN = 10
STEP = 1
input_words = []
label_words = []
text2=text.split()
for i in range(0,nb_words-SEQLEN,STEP):
    x=text2[i:(i+SEQLEN)]
    y=text2[i+SEQLEN]
    input_words.append(x)
    label_words.append(y)
print('input words list: ','\n',input_words[0])
print('label(output) words list: ','\n',label_words[0])

input words list:  
 ['project', 'gutenberg', 's', 'alice', 's', 'adventures', 'in', 'wonderland', 'by', 'lewis']
label(output) words list:  
 carroll


* Note that input_words is a list of lists and the output_words list is not.

Construct the vectors of the input and the output datasets.

In [8]:
total_words = len(set(words))
X = np.zeros((len(input_words), SEQLEN, total_words), dtype=np.bool)
y = np.zeros((len(input_words), total_words), dtype=np.bool)

I am creating empty arrays in the preceding step, which will be populated in the following code.

In [9]:
for i, input_word in enumerate(input_words):
    for j, word in enumerate(input_word):
        X[i, j, word2index[word]] = 1
    y[i,word2index[label_words[i]]]=1

In the preceding code 
* The first for loop is used to loop through all the words in the input sequence of words (10 words in input), and the second for loop is used to loop through an individual word in the chosen sequence of input words. 
* Additionally, given that the output is a list, I do not need to update it using the second for loop (as there is no sequence of IDs). 

The output shapes of X and y are as follows.

In [10]:
print('Shape of X: ',X.shape)
print('Shape of y: ',y.shape)

Shape of X:  (30527, 10, 3043)
Shape of y:  (30527, 3043)


Define the architecture of the model

In [11]:
HIDDEN_SIZE = 128
BATCH_SIZE = 32
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100
model = Sequential()
model.add(LSTM(HIDDEN_SIZE,return_sequences=False,input_shape=(SEQLEN,total_words)))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


A summary of the model is as follows.

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               1624064   
_________________________________________________________________
dense_1 (Dense)              (None, 3043)              392547    
Total params: 2,016,611
Trainable params: 2,016,611
Non-trainable params: 0
_________________________________________________________________


In [13]:
int(len(input_words)*0.1)

3052

In [14]:
len(input_words)

30527

### Fit the model. 
Look at how the output varies over an increasing number of epochs. Generate a random set of sequences of 10 words and try to predict the next possible word. We are in a position to observe how our predictions are getting better over an increasing number of epochs.

* In the following code, I am fitting my model on input and output arrays for one epoch. 
* I am choosing a random seed word (test_idx – which is a random number that is among the last 10% of the input array (as validation_split is 0.1) and I am collecting the input words at a random location. 
* I am converting the input sequence of IDs into a one-hot-encoded version (thus obtaining an array that is 1 x 10 x total_words in shape).
* Finally, making a prediction on the array I just created and obtain the word that has the highest probability.

In [15]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION, validation_split = 0.1)
    test_idx = np.random.randint(int(len(input_words)*0.1)) * (-1)
    test_words = input_words[test_idx]
    print("Generating from seed: %s" % (test_words))
    for i in range(NUM_PREDS_PER_EPOCH):        
        Xtest = np.zeros((1, SEQLEN, total_words))
        for i, ch in enumerate(test_words):
            Xtest[0, i, word2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]
        print(ypred,end=' ')
        test_words = test_words[1:] + [ypred]

Iteration #: 0
Instructions for updating:
Use tf.cast instead.
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 95s 3ms/step - loss: 6.2254 - val_loss: 8.2025
Generating from seed: ['right', 'to', 'prevent', 'you', 'from', 'copying', 'distributing', 'performing', 'displaying', 'or']
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the ==================================================
Iteration #: 1
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 90s 3ms/step - loss: 5.8673 - val_loss: 8.4716
Generating from seed: ['e', '8', 'or', '

said the king and she went on on her head to make it s coming and i m a little thing i ve said to the mock turtle but i m not so i can t you myself you know said the king and you are have you said the king who said to the jury who was not to do you you know you re been said the mock turtle but i m not up and then i m not i can t you said alice and i ve seen them a little thing said alice a deep tone ==================================================
Iteration #: 11
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 89s 3ms/step - loss: 2.5779 - val_loss: 10.4086
Generating from seed: ['or', 'other', 'form', 'any', 'alternate', 'format', 'must', 'include', 'the', 'full']
turtle chapter important the mock turtle is in a very tone with voice please your majesty said alice and as she went on her as she went on in the mock turtle in a great hurry and alice had been been of her feet and her feet down down up alice began to see the baby a

27474/27474 [==============================] - 83s 3ms/step - loss: 0.4961 - val_loss: 12.1843
Generating from seed: ['owns', 'a', 'united', 'states', 'copyright', 'in', 'these', 'works', 'so', 'the']
baby are out of the air as it is to it again child corner a tie of this there they couldn t seem upon nine nine book in it s pictures so far now at this moment and it away made it away but she was not a fancy that i had that it no be no doubt you may much it too far so much yet you know what he said to alice asked with a low voice it s not evidence remarks cried said alice and not her very repeat but said alice s a long dream of ==================================================
Iteration #: 22
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 79s 3ms/step - loss: 0.4046 - val_loss: 12.3187
Generating from seed: ['newby', 'chief', 'executive', 'and', 'director', 'gbnewby', 'pglaf', 'org', 'section', '4']
the hot s voice of the mock t

one end across they were other voice off which silence that cheered the dormouse had made of them of her own i so think to be herself this a good size off as together happened oh to speak to alice again a minute she began to made the next house she was having seen against a few of day just just heard a little sure but she s too back to the end chapter that the trial day in that s very got to alice but the opportunity for making quite quite out of sight and that i wish i ==================================================
Iteration #: 32
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 77s 3ms/step - loss: 0.0664 - val_loss: 13.1606
Generating from seed: ['of', 'this', 'agreement', 'for', 'keeping', 'the', 'project', 'gutenberg', 'tm', 'name']
the most turtle is alice i don t know his before she did not even down at the garden where poor see the dormouse happens in the earth again for there could be get to suddenly feet bread it wa

if they had manage enough when they had the kind of bread and quietly made off at feet and the dodo began change it things let the jury of it is the mock turtle is sing you said from father how to have the tarts the little for instance it s always different such in a pool and there with her time she was upon a cheshire more alice to feel in at very before but she had a little bottle that it s certainly indeed not said to alice why why it was a very rule after me and ==================================================
Iteration #: 44
Train on 27474 samples, validate on 3053 samples
Epoch 1/1
27474/27474 [==============================] - 199s 7ms/step - loss: 0.0088 - val_loss: 13.5365
Generating from seed: ['federal', 'laws', 'and', 'your', 'state', 's', 'laws', 'the', 'foundation', 's']
on on all the little word and there s too too close to stay down however she waited in her time close come at direction it might she found a very way of of after alice thought she could herself it and b